In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

dt = 0.001

def Force(x1, x2):
    r = x2 - x1
    N = r.shape[0]
    dist = tf.norm(r)
    dist = tf.reshape(dist, [N, N, 1])
    return r / (dist ** 3.0 + 1e-1)

def n_body():
    X = tf.input([-1, 2], tf.float32)
    N = X.shape[0]
    V = tf.input([N, 2], tf.float32)

    i, j, k = tf.indices([N, N, 2])
    
    Fij = Force(X[i, k], X[j, k])
    Fij = tf.select(i == j, 0.0, Fij)
    
    Fi = tf.sum(Fij, axis=1)

    Vnew = V + Fi * dt
    Xnew = X + Vnew * dt

    return [Xnew, Vnew]

nbody = tf.compile(n_body)

TensorFrost module loaded in debug mode! Expect slow performance.
n_body:
  Kernel count: 2
  Intermediate buffers: 1
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 418
  IR Compile time: 39.552101 ms
  Compiler time: 1299.874756 ms



In [6]:
import cv2 as cv

N = 2000
X = np.random.randn(N, 2).astype(np.float32)
#V = np.zeros([N, 2], np.float32)
#initialize with rotation
V = np.zeros([N, 2], np.float32)
V[:, 0] = -X[:, 1] * 5.1
V[:, 1] = X[:, 0] * 5.1

Xtf = tf.tensor(X)
Vtf = tf.tensor(V)

#file_path = 'H:/TestVideos/fluid.gif'
#writer = imageio.get_writer(file_path, fps=30, loop=0)

#use opencv to write is as video too
video_path = 'H:/TestVideos/sim.mp4'
fourcc = cv.VideoWriter_fourcc(*'H264')
video = cv.VideoWriter(video_path, fourcc, 60, (800, 800))

for i in range(200):
    Xtf, Vtf = nbody(Xtf, Vtf)

    X = Xtf.numpy
    V = Vtf.numpy

    #make scatter plot
    img = np.zeros([800, 800, 3], np.uint8)
    X = (X / 5) * 400 + 400
    X = np.clip(X, 0, 800-1)
    X = X.astype(np.int32)
    img[X[:, 0], X[:, 1]] = 255

    #write to video
    video.write(img)
    #writer.append_data(img)

#writer.close()
video.release()